In [ ]:
from telethon.sync import TelegramClient
from telethon.tl.functions.contacts import ResolveUsernameRequest
from telethon.tl.functions.channels import GetMessagesRequest
from telethon.tl.functions.messages import GetHistoryRequest, ReadHistoryRequest 
from telethon import functions, types
import datetime
from time import sleep
import os
import glob
from random import random
import re
import asyncio
from tqdm.notebook import tqdm
import pandas as pd 

### Этот блок нужно заполнить вашими значениями

In [ ]:
# Замените на ваши данные, брать на my.telegram.org
api_id, api_hash = open('../../tg_api_id.txt', 'r').read().split()

my_channel = 'etonikmedoed'

# Файл с вашим сообщением, поддерживается telegram-форматирование в формате html
message_file = "message.txt"

In [ ]:
client= TelegramClient('anon', api_id, api_hash)
await client.start()

In [6]:
def log_error(text):
    print(text)
    with open("log_err.txt", "a+", encoding="utf8") as f:
        f.write(f"{text}\n")   

In [ ]:
# получить все групповые чаты 
result = await  client(functions.messages.GetAllChatsRequest(except_ids=[]))
[i.title for i in result.chats]

In [ ]:
# все контакты
contacts = await client(functions.contacts.GetContactsRequest(hash=0))

In [ ]:
# контакты, с которыми есть в истории более 30 сообщений

users=[]
for user in tqdm(contacts.users):
    if (await client.get_messages(user, 10)).total>30:
        users.append(user)
    await asyncio.sleep(0.5)

In [ ]:
# Подписки на канале
result = await client(functions.channels.GetParticipantsRequest(
        channel=my_channel,
        filter=types.ChannelParticipantsRecent(),
        offset=0,
        limit=100,
        hash=0
    ))
channelusers = result.users
chuid = [u.id for u in channelusers]

In [ ]:
# Все отобранные выше контакты, что ещё не подписались, сохраняем в таблицу

db = pd.DataFrame.from_records([s.to_dict() for s in users if s.id not in chuid])
d = db[['first_name','last_name','username','phone',"id"]]
d.to_excel("to_invite.xlsx")

### Почистите таблицу руками от тех контактов, которых стесняетесь приглашать

In [ ]:
# Почистили таблицу руками, загружаем обратно
# ещё разок фильтруем от участников, 
# получаем список на приглашения

data =pd.read_excel('invite_ok.xlsx')
invite = data.loc[~data.id.isin(chuid), : ]

In [ ]:
print(invite.count())
invite

In [ ]:
# шлём всем приглашения всем из списка

for n, userl in enumerate(invite.values.tolist()):    
    message = open(message_file,"r", encoding="utf-8").read().split("=====")
    try:
        user = await client.get_entity(userl[-1])  
        notif = f"{user.first_name} {user.last_name} {user.username} {user.id}"
        print(f"start {notif}")

        await asyncio.sleep(4+random()*5)
        try:
            await client.send_message(user, message[0], parse_mode="html")
          
            with open("log_notificated.txt", "a+") as log_ok:
                log_ok.write(f"{notif}\n")
        except Exception as e:
            log_error(f"{n} - error, err: {e}")
    except:
        log_error(f"{n} - not found")
print("ok")